# Imports

In [1]:
import matplotlib.pyplot as plt
from os import path
from wordcloud import WordCloud
import nltk
import pandas as pd

### Example of usage

In [2]:
d = path.dirname('hillary-clinton-emails/')

# Read the whole text.
text = open(path.join(d, 'Emails.csv'),encoding='utf8').read()

# Generate a word cloud image
wordcloud = WordCloud().generate(text)

# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
plt.imshow(wordcloud)
plt.axis("off")

# lower max_font_size
wordcloud = WordCloud(max_font_size=40).generate(text)
plt.figure()
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [7]:
type(text)

str

# Read database

In [3]:
df=pd.read_csv('hillary-clinton-emails/Emails.csv')

In [4]:
df.shape

(7945, 22)

In [5]:
df.head(1)

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...


## Tokenization of RawText

In [6]:
df.ExtractedBodyTextString=df.ExtractedBodyText.apply(lambda x: str(x))

In [7]:
sentence="".join(df.ExtractedBodyTextString)

In [8]:
len(sentence)

3645959

In [9]:
tokens = nltk.word_tokenize(sentence)

In [10]:
len(tokens)

689100

In [11]:
text = nltk.Text(tokens)

In [12]:
text.collocations()

United States; White House; State Department; FOIA WAIVER; BENGHAZI
COMM; SELECT BENGHAZI; HOUSE SELECT; SENSITIVE INFORMATION;
F-2015-04841 Doc; STATE DEPT; 05/13/2015 STATE; Private Residence; New
York; *En route; State Case; OFFICE TIME; U.S. Department; health
care; Middle East; Conference Room


## Stop words removal

In [9]:
from nltk.corpus import stopwords

In [10]:
stop = stopwords.words('english')

In [14]:
cont=0
for i in tokens:
    if i in stop:
        cont+=1

In [15]:
cont

220707

In [16]:
good_tokens=[x for x in tokens if x not in stop]

In [17]:
len(good_tokens)

468393

In [18]:
text = nltk.Text(tokens)

In [20]:
text.collocations()

United States; White House; State Department; FOIA WAIVER; BENGHAZI
COMM; SELECT BENGHAZI; HOUSE SELECT; SENSITIVE INFORMATION;
F-2015-04841 Doc; STATE DEPT; 05/13/2015 STATE; Private Residence; New
York; *En route; State Case; OFFICE TIME; U.S. Department; health
care; Middle East; Conference Room


## Stemming

In [19]:
from nltk.stem.wordnet import WordNetLemmatizer

In [20]:
lmtzr = WordNetLemmatizer()

In [21]:
stemmed_tokens=[lmtzr.lemmatize(x) for x in good_tokens]

In [22]:
len(stemmed_tokens)

468393

In [23]:
text = nltk.Text(tokens)

In [26]:
text.collocations()

United States; White House; State Department; FOIA WAIVER; BENGHAZI
COMM; SELECT BENGHAZI; HOUSE SELECT; SENSITIVE INFORMATION;
F-2015-04841 Doc; STATE DEPT; 05/13/2015 STATE; Private Residence; New
York; *En route; State Case; OFFICE TIME; U.S. Department; health
care; Middle East; Conference Room


## Wordcloud

In [27]:
from wordcloud import WordCloud

In [25]:
final_string=' '.join(stemmed_tokens)

In [26]:
# Generate a word cloud image
wordcloud = WordCloud().generate(final_string)

# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
plt.imshow(wordcloud)
plt.axis("off")

# lower max_font_size
wordcloud = WordCloud(max_font_size=40).generate(final_string)
plt.figure()
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

# LDA

Import at first the cleaned body text from a csv file:

In [2]:
df = pd.read_csv('hillary-clinton-emails/sentimentEmails.csv')

We take a look at the dataset before proceeding:

In [3]:
df.head(10)

,ExtractedBodyText,MetadataSubject,Sentiment
0,"B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: La...",H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,0.0000
1,Thx,CHRIS STEVENS,0.3612
2,"H <hrod17@clintonemail.com>\r\nFriday, March 1...",H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,0.0000
3,Pis print.\r\n-•-...-^\r\nH < hrod17@clintoner...,MEET THE RIGHT-WING EXTREMIST BEHIND ANTI-MUSL...,-0.9191
4,"H <hrod17@clintonemail.corn>\r\nFriday, March ...",H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,0.0000
5,FYI,SECRETARY'S REMARKS,0.2023
6,"B6\r\nWednesday, September 12, 2012 6:16 PM\r\...",MORE ON LIBYA,0.0000
7,Fyi\r\nB6\r\n— —,ABZ AN HBJ ON LIBYA AND WEST BANK/GAZA,0.2023
8,"B6\r\nWednesday, September 12, 2012 6:16 PM\r\...",MORE ON LIBYA,0.0000
9,Fyi,HEY,0.2023


At first we define a list of stopwords to be wiped out from the documents; this words are typical English language stopwords or some trivial expressions which are contained in the *RawText*, such as 'fw:', 'sent:', 'from:', 'to:' (mail parameters) or 'u.s.', 'state', 'department' and so on:

In [28]:
# clear the documents from trivial recurrent words
# NOTE: 08/31/2015 is a recurrent date
# f-2014-20439 is related to the department identification code or something similar
stop = stopwords.words('english')

trivial_topics = ['date:','unclassified','from:','sent:','subject:','to:','state','department',
                  'no.','doc','case','u.s.','fw:','f-2014-20439','08/31/2015',
                 're:','cc:','h','j']

punctuation_symbols = ", : ; . ' % & $ @ - ( ) — <  > • * / + -".split()

# add punctuation symbols to the stopwords into a new list called stoplist
stoplist = list(set(stop).union(set(punctuation_symbols).union(trivial_topics)))

# apply the stoplist to each document in RawText
documents = [[word for word in text.lower().split() if word not in stoplist]
            for text in df.ExtractedBodyText.dropna()]

Now we import the *gensim* library and define a **dictionary**, which matches any word in each text with a numeric ID; notice that the documents are treated as *bows* (numeric vectors); the output of this operation is the **corpus** we will perform analysis on:

In [29]:
# define a dictionary to associate ad Id to each token and build the corpus
from gensim import corpora, models
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(text) for text in documents]

C:\Users\Utente\Anaconda3\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Utente\Anaconda3\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


Now we define the Latent Dirichlet Allocation using the dictionary and the corpus. Num_topics = **20** seems a reasonable choice:

In [30]:
# define an lda model using the previously defined dictionary
no_topics = 20
lda = models.ldamodel.LdaModel(corpus, id2word = dictionary, num_topics=no_topics)

We perform now some technical operations to print the topics in a sufficiently readable way:

In [31]:
topics = []
for num in range(no_topics):
    topic_prob = lda.show_topic(num)
    topic = []
    for word in range(len(topic_prob)):
        topic.append(topic_prob[word][0])
    topic = '  '.join(topic)
    topics.append(topic)
topics

['israeli  book  israel  iraqi  richards  one  called  referring  recommend  obama',
 "you'll  cloture  it?  agree  new  kerry  office  2010  pm  call",
 'new  would  american  one  people  united  president  obama  government  said',
 'fyi  b6  thought  afghan  (202)  wjc  see  dc  <verveerms@state.gov>  book,',
 'netanyahu  cameron  mr  israeli  russia  it.  miliband  speech  2010  david',
 'prince  2010  also  part  sullivan,  sent  new  would  via  b6',
 'hikers  fyi.  mr.  would  print  me.  missile  pls  ashton  nuclear',
 'mr.  talks  would  know  call  could  sunday,  ■  give  pm',
 'see  w  call  would  jake  could  know  pm  boehner  thanks',
 'kurdish  new  would  f.  haitian  see  white  taliban  said  may',
 "pm  secretary's  office  meeting  room  conference  private  route  depart  arrive",
 "pm  office  secretary's  arrive  route  depart  meeting  residence  private  *en",
 'people  red  new  one  women  american  even  work  president  would',
 '2010  b1  1.4(d)  chery